<a href="https://www.kaggle.com/code/aleksandrmorozov123/recommender-systems?scriptVersionId=190623638" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv


**Collaborative filtering**

In [20]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random
from IPython.display import Image
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.prediction_algorithms import CoClustering
from surprise.prediction_algorithms import NMF
from surprise import accuracy

# read the data
data = pd.read_csv ('/kaggle/input/ecommerce-data/Order Details.csv')
data.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


In [21]:
# Data Info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB


In [22]:
data.isnull().sum().sort_values(ascending=False)

Order ID        0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64

In [23]:
data1 = data.dropna()
data1.describe()

,Amount,Profit,Quantity
count,1500.000000,1500.000000,1500.000000
mean,287.668000,15.970000,3.743333
std,461.050488,169.140565,2.184942
min,4.000000,-1981.000000,1.000000
25%,45.000000,-9.250000,2.000000
50%,118.000000,9.000000,3.000000
75%,322.000000,38.000000,5.000000
max,5729.000000,1698.000000,14.000000


In [30]:
# creating the data matrrix about purchasing history
purchase_df = (data1.groupby(['Order ID', 'Category'])['Quantity'].
               sum().unstack().reset_index().fillna(0).set_index('Order ID'))
purchase_df.head()

Category,Clothing,Electronics,Furniture
Order ID,,,
B-25601,8.0,4.0,7.0
B-25602,11.0,11.0,0.0
B-25603,24.0,0.0,1.0
B-25604,11.0,0.0,0.0
B-25605,7.0,0.0,0.0


In [31]:
# encoding the matrix
def encode_units(x):
    if x < 1: # If the quantity is less than 1
        return 0 # Not purchased
    if x >= 1: # If the quantity is greater than 1
        return 1 # Purchased
    
purchase_df = purchase_df.map(encode_units)
purchase_df.head()

Category,Clothing,Electronics,Furniture
Order ID,,,
B-25601,1,1,1
B-25602,1,1,0
B-25603,1,0,1
B-25604,1,0,0
B-25605,1,0,0


In [34]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, B-25601 to B-26100
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Clothing     500 non-null    int64
 1   Electronics  500 non-null    int64
 2   Furniture    500 non-null    int64
dtypes: int64(3)
memory usage: 31.8+ KB


****Similarity measures****

In [35]:
# apply cosine similarity
user_similarities = cosine_similarity(purchase_df)
user_similarity_data = pd.DataFrame(user_similarities,
                                    index = purchase_df.index,columns=purchase_df.index)
user_similarity_data.head()

Order ID,B-25601,B-25602,B-25603,B-25604,B-25605,B-25606,B-25607,B-25608,B-25609,B-25610,...,B-26091,B-26092,B-26093,B-26094,B-26095,B-26096,B-26097,B-26098,B-26099,B-26100
Order ID,,,,,,,,,,,,,,,,,,,,,
B-25601,1.000000,0.816497,0.816497,0.577350,0.577350,0.577350,0.577350,1.000000,0.816497,1.000000,...,0.577350,0.577350,1.000000,0.577350,0.577350,0.816497,1.000000,1.000000,0.816497,0.816497
B-25602,0.816497,1.000000,0.500000,0.707107,0.707107,0.707107,0.707107,0.816497,1.000000,0.816497,...,0.707107,0.707107,0.816497,0.707107,0.707107,1.000000,0.816497,0.816497,1.000000,0.500000
B-25603,0.816497,0.500000,1.000000,0.707107,0.707107,0.707107,0.707107,0.816497,0.500000,0.816497,...,0.707107,0.707107,0.816497,0.707107,0.707107,0.500000,0.816497,0.816497,0.500000,1.000000
B-25604,0.577350,0.707107,0.707107,1.000000,1.000000,1.000000,1.000000,0.577350,0.707107,0.577350,...,1.000000,1.000000,0.577350,1.000000,1.000000,0.707107,0.577350,0.577350,0.707107,0.707107
B-25605,0.577350,0.707107,0.707107,1.000000,1.000000,1.000000,1.000000,0.577350,0.707107,0.577350,...,1.000000,1.000000,0.577350,1.000000,1.000000,0.707107,0.577350,0.577350,0.707107,0.707107


In [37]:
# create function for getting recommendations 

def fetch_similar_users(user_id,k=5):
    # separating data rows for the entered user id
    user_similarity = user_similarity_data[user_similarity_data.index == user_id]
    # a data of all other users
    other_users_similarities = user_similarity_data[user_similarity_data.
    index != user_id]
    # calcuate cosine similarity between user and each other user
    similarities = cosine_similarity(user_similarity,other_users_similarities)[0].tolist()
    user_indices = other_users_similarities.index.tolist()
    index_similarity_pair = dict(zip(user_indices, similarities))
    # sort by similarity
    sorted_index_similarity_pair = sorted(index_similarity_pair.items(),reverse = True)
    top_k_users_similarities = sorted_index_similarity_pair[:k]
    similar_users = [u[0] for u in top_k_users_similarities]
    print('The users with behaviour similar to that of user {0} are:'.format(user_id))
    return similar_users

similar_users = fetch_similar_users(1)
similar_users

ValueError: Found array with 0 sample(s) (shape=(0, 500)) while a minimum of 1 is required by check_pairwise_arrays.

In [10]:
# item-to-item collaborative filtering

items_purchase_df = (data1.groupby(['Clothing ID', 'Class Name'])['Rating'].
sum().unstack().reset_index().fillna(0).set_index('Clothing ID'))
items_purchase_df.head()

Class Name,Blouses,Casual bottoms,Chemises,Dresses,Fine gauge,Intimates,Jackets,Jeans,Knits,Layering,Legwear,Lounge,Outerwear,Pants,Shorts,Skirts,Sleep,Sweaters,Swim,Trend
Clothing ID,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
items_purchase_df = items_purchase_df.map(encode_units)
item_similarities = cosine_similarity(items_purchase_df)
item_similarity_data = pd.DataFrame(item_similarities,
                                    index=items_purchase_df.index,columns=items_purchase_df.index)
item_similarity_data.head()

Clothing ID,1,2,4,5,7,8,9,10,11,12,...,1193,1195,1197,1198,1199,1200,1202,1203,1204,1205
Clothing ID,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**K-Nearest neighbor**

In [12]:
purchase_matrix = csr_matrix(purchase_df.values)
knn_model = NearestNeighbors(metric = 'euclidean', algorithm = 'brute')
knn_model.fit(purchase_matrix)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [13]:
def fetch_similar_users_knn(purchase_df,query_index):
    # Creating empty list where we will store user id of similar users
    simular_users_knn = []
    # Storing the distance and index of nearest neighbor
    distances, indices = knn_model.kneighbors(purchase_df.iloc[
        query_index,:].values.reshape(1, -1), n_neighbors = 5)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(purchase_df.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, purchase_df.index[indices.flatten()[i]], distances.flatten()[i]))
            simular_users_knn.append( purchase_df.index[indices.
            flatten()[i]])
            
fetch_similar_users_knn(purchase_df,16)

Recommendations for 19:

1: 1132, with distance of 0.0:
2: 1129, with distance of 0.0:
3: 1130, with distance of 0.0:
4: 1126, with distance of 0.0:


In [14]:
data3 = items_purchase_df.stack().to_frame()
#Renaming the column as Quantity
data3 = data3.reset_index().rename(columns={0:"Quantity"})
data3

,Clothing ID,Class Name,Quantity
0,1,Blouses,0
1,1,Casual bottoms,0
2,1,Chemises,0
3,1,Dresses,0
4,1,Fine gauge,0
...,...,...,...
21895,1205,Skirts,0
21896,1205,Sleep,0
21897,1205,Sweaters,0
21898,1205,Swim,0


In [15]:
print(items_purchase_df.shape)
print(data3.shape)

(1095, 20)
(21900, 3)


In [17]:
# Storing all customer ids in customers
customer_ids = data1['Clothing ID']

# Storing all item descriptions in items
item_ids = data1['Class Name'] 

In [19]:
from collections import Counter

# counting no. of orders made by each customer
count_orders = Counter(customer_ids)

# storing the count and customer id in a dataframe
customer_count_df = pd.DataFrame.from_dict(count_orders, orient='index').reset_index().rename(columns={0:"Quantity"})